In [223]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from torch.utils.data import DataLoader

import torchvision.transforms as transforms
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

import os

In [224]:
from scipy.io import loadmat

In [225]:
#configuring the device
device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')

In [226]:
# hyperparameters
batch_size = 32
num_epochs = 50
learning_rate = 0.001

In [227]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [228]:
#dataset root directory
data_dir = 'dataset_flower102/'

In [229]:
#load train data
train_dataset = torchvision.datasets.Flowers102(root= data_dir, split = 'train', transform = data_transforms["train"], download = True)

#load test data
test_dataset = torchvision.datasets.Flowers102(root= data_dir, split = 'test', transform = data_transforms["test"], download = True)

#load valid
valid_dataset = torchvision.datasets.Flowers102(root= data_dir, split = 'val', transform = data_transforms["val"], download = True)

In [230]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)

In [231]:
#implement conv net
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #first convolutional layer
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        
        #pooling layer
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        
        #second convolutional layer
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels=32, kernel_size=(3,3), stride = (1,1), padding = (1,1))
        
        #fully connected layers
        self.fc1 = nn.Linear(32 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 102)
        
        #dropout layer
        self.dropout = nn.Dropout(p=0.5)


    def forward(self,x):
        
        #convolution + RelU + Pooling layer 1
        x = self.pool(F.relu(self.conv1(x)))
        
        #convolutional + ReLU + Pooling layer 2
        x = self.pool(F.relu(self.conv2(x)))
        
        
        
        #flatten the tensor for fully connected layer
        #x = x.view(-1, 32 * 56 * 56)
        x = x.view(x.size(0),-1)
        
        #fully connected layer 1 with ReLU and Dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(x)
        
        #fully connected layer 2
        x = self.fc2(x)
        
        return x

In [232]:
model = CNN().to(device)

In [233]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [234]:
n_steps_total = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    for i, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        
        #forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #optimize
        optimizer.step()
        
        if (i + 1) % 8 == 0:
            print(f"Epoch [{epoch+ 1}/{num_epochs}], Step[{i+1}/{n_steps_total}], Loss: {loss.item():.4f}")
        

Epoch [1/50], Step[8/32], Loss: 5.8847
Epoch [1/50], Step[16/32], Loss: 4.6363
Epoch [1/50], Step[24/32], Loss: 4.6059
Epoch [1/50], Step[32/32], Loss: 4.5471
Epoch [2/50], Step[8/32], Loss: 4.6163
Epoch [2/50], Step[16/32], Loss: 4.5721
Epoch [2/50], Step[24/32], Loss: 4.5494
Epoch [2/50], Step[32/32], Loss: 4.4520
Epoch [3/50], Step[8/32], Loss: 4.3033
Epoch [3/50], Step[16/32], Loss: 4.5609
Epoch [3/50], Step[24/32], Loss: 4.5950
Epoch [3/50], Step[32/32], Loss: 4.5842
Epoch [4/50], Step[8/32], Loss: 4.4051
Epoch [4/50], Step[16/32], Loss: 4.5180
Epoch [4/50], Step[24/32], Loss: 4.0175
Epoch [4/50], Step[32/32], Loss: 4.3948
Epoch [5/50], Step[8/32], Loss: 4.3085
Epoch [5/50], Step[16/32], Loss: 4.2646
Epoch [5/50], Step[24/32], Loss: 4.2753
Epoch [5/50], Step[32/32], Loss: 4.2828
Epoch [6/50], Step[8/32], Loss: 3.8649
Epoch [6/50], Step[16/32], Loss: 4.1342
Epoch [6/50], Step[24/32], Loss: 4.1774
Epoch [6/50], Step[32/32], Loss: 4.2486
Epoch [7/50], Step[8/32], Loss: 3.8819
Epoch [

In [235]:
torch.save(model.state_dict(),"flowers-102.pth")

In [236]:
#initialize correct predictions,valid loss and total samples counter to 0
def evaluate_model(model, test_loader):
    model.eval()
    n_correct = 0
    n_samples = 0
    with torch.no_grad():
        for data, targets in valid_loader:
            #move data and targets to the appropriate device
            data = data.to(device)
            targets = targets.to(device)
        
            #Forward pass: compute predfictions
            scores = model(data)
        
            #get the predicted class with the highest score
            _, predicted = torch.max(scores, 1)
        
            #update the count of correct prediction
            n_correct += predicted.eq(targets).sum().item()
        
            #update the count of total samples
            n_samples += targets.size(0) 
            
    accuracy = 100.0 *n_correct / n_samples
    print(f'Got {n_correct} / {n_samples} with accuracy   {accuracy:.2f}%')

    

In [237]:
evaluate_model(model, test_loader)

Got 170 / 1020 with accuracy   16.67%
